# Focus Radargram
## 0. Imports

In [ ]:
from surface import *
from source import *
from model import *

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from math import floor
from math import ceil

import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

## 1. Load radargram

In [ ]:
rdrgrm = np.load("rdrgrm.npy")

In [ ]:
# radargram params
st_t = 166.8e-6   # start
en_t = 175e-6  # end
N = rdrgrm.shape[0]      # how many "range bins?"
n = rdrgrm.shape[1]      # how many traces
t_bin = (en_t-st_t) / N

In [ ]:
fig = px.imshow(np.real(rdrgrm), aspect="auto", color_continuous_scale='gray', width=800, height=600)
ticktext = ['{0:5.1f}'.format(val) for val in np.arange(st_t*1e6, en_t*1e6, 1)]
tickvals = np.linspace(0, N, len(ticktext))

fig.update_yaxes(
    tickvals=tickvals,
    ticktext=ticktext,
    title_text="Time (µs)"
)
fig.show()

## 2. Range Compression

In [ ]:
rc = np.zeros_like(rdrgrm)

# generate sample source to cross correlate with
source = Source(1e-9, 0.5e-6, (0, 0, 0))
source.chirp(9e6, 1e6)
signal = source.signal

for i in range(n):
    rc[:,i] = np.correlate(rdrgrm[:,i], signal, mode='same')

In [ ]:
rc.shape

In [ ]:
fig = px.imshow(np.real(rc), aspect="auto", color_continuous_scale='gray', width=800, height=600)
ticktext = ['{0:5.1f}'.format(val) for val in np.arange(st_t*1e6, en_t*1e6, 1)]
tickvals = np.linspace(0, N, len(ticktext))
fig.update_xaxes(title_text="Trace #")
fig.update_yaxes(tickvals=tickvals, ticktext=ticktext, title_text="Time (µs)")
fig.show()

## 3. Azumith FFT & Delay Estimation

In [ ]:
az_fft = np.fft.fft(rc, axis=1)
az_freq = np.fft.fftshift(np.fft.fftfreq(n, d=source.dt))
#az_freq = np.fft.fftfreq(n, d=source.dt)
az_fftshift = np.fft.fftshift(az_fft, axes=1)

In [ ]:
# delay estimation
az_max = np.argmax(az_fftshift, axis=0)

In [ ]:
fig = px.imshow(np.real(az_fftshift), aspect="auto", color_continuous_scale='gray', 
                zmin=-5000, zmax=5000, width=800, height=600)

ticktext = ['{0:5.1f}'.format(val) for val in np.arange(st_t*1e6, en_t*1e6, 1)]
tickvals = np.linspace(0, N, len(ticktext))
axtext = ['{0:1f}'.format(v/(1e6)) for v in np.interp(np.linspace(0, n, 21), range(n), az_freq)]
axvals = np.linspace(0, n, len(axtext))

fig.add_trace(go.Scatter(
    x=list(range(n)), 
    y=az_max,
    mode='lines',
    name='Delay Estimate',
    line=dict(color='red')
))

fig.update_xaxes(tickvals=axvals,   ticktext=axtext,   title_text="Doppler Domain - Freq (MHz)")
fig.update_yaxes(tickvals=tickvals, ticktext=ticktext, title_text="Time (µs)")
fig.show()

## 4. PF RCMC (Point Facet Range Cell Migration Correction)

In [ ]:
timebuff = int(az_fftshift.shape[0]/4)
rcmc = np.zeros((int(2*timebuff), n)).astype(np.complex128)
for i, az in enumerate(az_max):
    tmin, tmax = max(az-timebuff, 0), min(az_fftshift.shape[0], az+timebuff)
    segment = az_fftshift[tmin:tmax, i]
    if tmin == 0:
        segment = np.concatenate((np.zeros(int(timebuff*2-len(segment))), segment))
    elif tmax == az_fftshift.shape[0]:
        segment = np.concatenate((segment,np.zeros(int(timebuff*2-len(segment)))))
    rcmc[:len(segment), i] = segment

In [ ]:
fig = px.imshow(np.real(rcmc), aspect="auto", color_continuous_scale='gray', 
                zmin=-5000, zmax=5000, width=800, height=300)
r1 = timebuff / az_fft.shape[0]
ticktext = ['{0:5.1f}'.format(val) for val in np.arange(r1*(en_t-st_t)*-1e6, (1-r1)*(en_t-st_t)*1e6, 1)]
tickvals = np.linspace(0, N, len(ticktext))
axtext = ['{0:1f}'.format(v/(1e6)) for v in np.interp(np.linspace(0, n, 21), range(n), az_freq)]
axvals = np.linspace(0, n, len(axtext))

fig.add_trace(go.Scatter(
    x=list(range(n)), 
    y=np.zeros(n)+timebuff,
    mode='lines',
    name='Delay Estimate',
    line=dict(color='red')
))

fig.update_xaxes(tickvals=axvals,   ticktext=axtext,   title_text="Doppler Domain - Freq (MHz)")
fig.update_yaxes(tickvals=tickvals, ticktext=ticktext, title_text="Time (µs)")
fig.show()

## 5. Extract 1D Reference Function

In [ ]:
reffun = rcmc[timebuff, :]

In [ ]:
df = pd.DataFrame({'Doppler Domain - Freq (Hz)':np.interp(np.linspace(0, n, n), range(n), az_freq), 'Power (dB)':10*np.log10(np.abs(reffun))})
fig = px.line(df, x='Doppler Domain - Freq (Hz)', y='Power (dB)')
fig.show()

## 6. Azumith Compression

In [ ]:
azcmp = np.zeros_like(rcmc)

"""
# compute fft of reference function
fft_source = fft(reffun)

for i in range(rcmc.shape[0]):
    fft_trace = fft(rcmc[i, :])
    cc_freq = fft_source * np.conj(fft_trace)
    cc_time = ifft(cc_freq)
    cc_time = np.fft.fftshift(cc_time)
    azcmp[i,:] = np.flip(np.real(cc_time))
"""

for i in range(rcmc.shape[0]):
    azcmp[i,:] = np.convolve(reffun, rcmc[i,:], mode='same')

azcmp /= np.max(azcmp)

In [ ]:
fig = px.imshow(np.real(azcmp), aspect="auto", color_continuous_scale='gray', 
                zmin=-1, zmax=1, width=800, height=300)
r1 = timebuff / az_fft.shape[0]
ticktext = ['{0:5.1f}'.format(val) for val in np.arange(r1*(en_t-st_t)*-1e6, (1-r1)*(en_t-st_t)*1e6, 1)]
tickvals = np.linspace(0, N, len(ticktext))
axtext = ['{0:1f}'.format(v/(1e6)) for v in np.interp(np.linspace(0, n, 21), range(n), az_freq)]
axvals = np.linspace(0, n, len(axtext))

fig.update_xaxes(tickvals=axvals,   ticktext=axtext,   title_text="Doppler Domain - Freq (MHz)")
fig.update_yaxes(tickvals=tickvals, ticktext=ticktext, title_text="Time (µs)")
fig.show()

## 7. Azumith IFFT

In [ ]:
azcmp_shift = np.fft.ifftshift(azcmp, axes=1)
focused = np.fft.ifft(azcmp_shift, axis=1)

In [ ]:
fig = px.imshow(np.real(focused), aspect="auto", color_continuous_scale='gray', width=800, height=600)
ticktext = ['{0:5.1f}'.format(val) for val in np.arange(st_t*1e6, en_t*1e6, 1)]
tickvals = np.linspace(0, N, len(ticktext))
fig.update_xaxes(title_text="Trace #")
fig.update_yaxes(tickvals=tickvals, ticktext=ticktext, title_text="Time (µs)")
fig.show()